# SQL DATA ANALYSIS TASK

# Context
Fenix is a company that is in renewable energy. To this end, it sells solar kits to populations who are in need. Customers have the option of paying for the kit in cash or on credit. If the customer chooses to buy on credit, he can choose between one year (360 days) or two years (720). He is then asked to pay an amount per day (daily_rate), this amount depending on the duration of the loan and the type of kit he has chosen and corresponds to a day of light. So every time he pays that amount, he has a day of light. After 24 hours the system hangs and it has to make another payment to take advantage of the light. He can decide to pay for more than one day of light. The day number is then the whole part of the amount / daily rate ratio. After this number of days, the system still locks. When a customer spends more than 30 days without light, he is considered inactive. Paid vs Expected (PvE) means the ratio between the total amount paid by the client and the amount he should have paid up to date. It allows to appreciate the recovery.

Let's consider the following tables

Loan_current_daitails

loan_id	|customer_id|daily_rate|days_elapsed|Days_til_lock_out|Total_paid
--------|-----------|----------|------------|-----------------|----------
loan_id_1|customer_id_1|daily_rate_1|days_elapsed_1|Days_til_lock_out_1|Total_paid_1
loan_id_2|customer_id_2|daily_rate_2|days_elapsed_2|Days_til_lock_out_2|Total_paid_2
.........|.............|............|..............|...................|............
loan_id_n|customer_id_n	|daily_rate_n|days_elapsed_n|Days_til_lock_out_n|Total_paid_n


Person_democraphic

customer_id	|country|	region|	profession
------------|-------|---------|-----------
customer_id_1|	Country_1	|Region_1	|profession_1
.............|	............|..........	|............
customer_id_j|	Country_j|	Region_j	|profession_j


Sales_details

loan_id	|Product_type	|Date_fullfiled	|loan_duration
--------|----------------|--------------|-------------
loan_id_1	|Product_type_1	|Date_fullfiled_1	|loan_duration_1
...|	...	|...	|...
loan_id_n|	Product_type_n|	Date_fullfiled_n|	loan_duration_n

### Tables description

* **Loan_id:** unique loan identifier

* **Customer_id:** unique customer identifier

* **Days_elapsed:** the number of days the client spends with us

* **Daily_rate:** expected amount from customer per day

* **Days_til_lock_out:** If positive, then corresponds to the number of days of light remaining before the system hangs. If negative, corresponds to the number of days since the system was blocked (the client has been without light since this number of days)

* **Total_paid:** total amount to be paid by the customer

* **Country:** Customer's country

* **Region:** customer department

* **Profession:** client's profession

* **Product_type:** the type of kit purchased by the customer

* **Loan_duration:** the duration of the customer's loan (360 or 720 days)

* **Date_fullfiled:** the date of the sale

### The datasets
The datasets exist in csv formats and can be downloaded and saved on disk

In [1]:
%load_ext sql
%sql sqlite:///assigndb.sqlite

In [2]:
%%sql
DROP TABLE IF EXISTS Loan_current_daitails;
DROP TABLE IF EXISTS Person_democraphic;
DROP TABLE IF EXISTS Sales_details;

 * sqlite:///assigndb.sqlite
Done.
Done.
Done.


[]

In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("assigndb.sqlite")
customers = pd.read_csv('Loan_current_daitails.csv')
orders = pd.read_csv('Person_democraphic.csv')
salesman = pd.read_csv('Sales_details.csv')

customers.to_sql("Loan", conn, index=False, if_exists="replace")
orders.to_sql("Person", conn, index=False, if_exists="replace")
salesman.to_sql("Sales", conn, index=False, if_exists="replace")

C:\Users\user\anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


### Write a SQL code to determine the total number of loans on the one hand and the total number of customers on the other

In [4]:
%%sql

SELECT COUNT(loan_id) Total_Num_of_Loans , COUNT(DISTINCT(customer_id)) Total_num_of_Cust 
FROM Loan;

 * sqlite:///assigndb.sqlite
Done.


Total_Num_of_Loans,Total_num_of_Cust
5000,4979


### Write a SQL code to determine the total number of loans by type of kit

In [5]:
%%sql

SELECT product_type PRODUCT, COUNT(loan_id) COUNT FROM Sales 
GROUP BY product_type;

 * sqlite:///assigndb.sqlite
Done.


PRODUCT,COUNT
Fenix 2,841
Fenix 3,974
Fenix Power 2+ Tondeuse,18
Fenix Power 4 + Woofer BJ,35
Fenix Power TV 19in FTA BJ,14
"Fenix Power TV 19in FTA Upgrade, BJ",28
Fenix Power TV 24in FTA BJ,634
"Fenix Power TV 24in FTA Upgrade, BJ",177
Fenix Radio +3,1791
Fenix Radio +5,422


### Write SQL code to determine the number of inactive loans

In [7]:
%%sql

SELECT COUNT(loan_id) INACTIVE_LOANS FROM Loan
WHERE days_til_lockout < -30;

 * sqlite:///assigndb.sqlite
Done.


INACTIVE_LOANS
626


### Write a SQL code to determine the regions where the total amount of payments exceeds 1,000,000

In [8]:
%%sql

SELECT person.region FROM person
LEFT JOIN Loan ON person.customer_id = Loan.customer_id
GROUP BY person.region HAVING SUM(Loan.total_paid) > 1000000;

 * sqlite:///assigndb.sqlite
Done.


region
Alibori
Atacora
Atlantique
Borgou
Collines
Couffo
Donga
Littoral
Mono
Oueme


### Write a SQL code to classify by region and in ascending order, the loans according to the total amount paid

In [9]:
%%sql

SELECT Loan.loan_id, person.region FROM person 
LEFT JOIN Loan ON person.customer_id = Loan.customer_id
GROUP BY person.region
ORDER BY person.region , Loan.total_paid  ASC;

 * sqlite:///assigndb.sqlite
Done.


loan_id,region
586196,Alibori
546198,Atacora
552403,Atlantique
565181,Borgou
617665,Collines
447804,Couffo
560165,Donga
598490,Littoral
798485,Mono
589556,Oueme



### Write a SQL code to determine the top 10 occupations by region with the highest total amount of payments

In [10]:
%%sql

SELECT person.region, Loan.loan_id , person.Profession FROM person 
LEFT JOIN Loan ON person.customer_id = Loan.customer_id
ORDER BY region;

 * sqlite:///assigndb.sqlite
Done.


region,loan_id,Profession
Alibori,586196,Doctor
Alibori,563052,Statistian
Alibori,839289,Nurse
Alibori,532670,Doctor
Alibori,593579,Ingenieur
Alibori,586231,Cmmercial
Alibori,900977,Tehnician
Alibori,563946,Nurse
Alibori,568692,Driver
Alibori,669452,Ingenieur
